In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
DROP Table IF EXISTS Person;
DROP Table IF EXISTS Drug;
DROP Table IF EXISTS DrugUse;

In [3]:
CREATE TABLE Person (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

CREATE TABLE Drug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(100) NOT NULL,
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

CREATE TABLE DrugUse (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId)
    REFERENCES Person(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Drug(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

In [4]:
SELECT * FROM Person;
SELECT * FROM Drug;
SELECT * FROM DrugUse

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT DISTINCT D.Name, COUNT(*)
    FROM Drug D, Person P, DrugUse DU
    WHERE D.Code=DU.DrugCode AND P.Id=DU.PersonId
    GROUP BY D.Name

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [6]:
CREATE VIEW ContagemCodigoMedicamento AS
SELECT DISTINCT D.Code, COUNT(*) QtdPessoas
    FROM Drug D, Person P, DrugUse DU
    WHERE D.Code=DU.DrugCode AND P.Id=DU.PersonId
    GROUP BY D.Code

In [7]:
SELECT CCM.Code FROM ContagemCodigoMedicamento CCM
                WHERE CCM.QtdPessoas=(SELECT MAX(QtdPessoas) FROM ContagemCodigoMedicamento)

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [8]:
SELECT RES.Code FROM (SELECT DISTINCT D.Code, COUNT(*) QtdPessoas
                            FROM Drug D, Person P, DrugUse DU
                            WHERE D.Code=DU.DrugCode AND P.Id=DU.PersonId
                            GROUP BY D.Code
                            ORDER BY QtdPessoas DESC) RES
WHERE ROWNUM=1

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [9]:
CREATE VIEW ContagemDecrescente AS
SELECT DISTINCT D.Code, COUNT(*) QtdPessoas
                            FROM Drug D, Person P, DrugUse DU
                            WHERE D.Code=DU.DrugCode AND P.Id=DU.PersonId
                            GROUP BY D.Code
                            ORDER BY QtdPessoas DESC

In [10]:
SELECT D.Name FROM Drug D WHERE D.Code=(SELECT CD.Code FROM ContagemDecrescente CD
WHERE ROWNUM=1)

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [11]:
SELECT AVG(RES.QtdMedicamento) FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU
GROUP BY PersonId) RES

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [12]:
CREATE VIEW ContagemPessoasMedicamento AS
SELECT RES.PersonId, RES.QtdMedicamento FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU
GROUP BY DU.PersonId) RES 

In [13]:
SELECT CPM.PersonId FROM ContagemPessoasMedicamento CPM
WHERE CPM.QtdMedicamento > (SELECT AVG(QtdMedicamento) FROM ContagemPessoasMedicamento)

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [14]:
CREATE VIEW GrupoIdade1 AS
SELECT AVG(RES.QtdMedicamento) MediaMedicamento FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU, Person P
WHERE DU.PersonId=P.Id AND P.Age<16 AND P.Age>=0
GROUP BY DU.PersonId) RES

In [15]:
CREATE VIEW GrupoIdade2 AS
SELECT AVG(RES.QtdMedicamento) MediaMedicamento FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU, Person P
WHERE DU.PersonId=P.Id AND P.Age<32 AND P.Age>=16
GROUP BY DU.PersonId) RES

In [16]:
CREATE VIEW GrupoIdade3 AS
SELECT AVG(RES.QtdMedicamento) MediaMedicamento FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU, Person P
WHERE DU.PersonId=P.Id AND P.Age<48 AND P.Age>=32
GROUP BY DU.PersonId) RES

In [17]:
CREATE VIEW GrupoIdade4 AS
SELECT AVG(RES.QtdMedicamento) MediaMedicamento FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU, Person P
WHERE DU.PersonId=P.Id AND P.Age<63 AND P.Age>=48
GROUP BY DU.PersonId) RES

In [18]:
CREATE VIEW GrupoIdade5 AS
SELECT AVG(RES.QtdMedicamento) MediaMedicamento FROM (SELECT DU.PersonId, COUNT(*) QtdMedicamento
FROM DrugUse DU, Person P
WHERE DU.PersonId=P.Id AND P.Age<81 AND P.Age>=63
GROUP BY DU.PersonId) RES

In [19]:
SELECT G1.MediaMedicamento FROM GrupoIdade1 G1 /* Idade = [0,15] */

2

In [20]:
SELECT G2.MediaMedicamento FROM GrupoIdade2 G2 /* Idade = [16,31] */

2

In [21]:
SELECT G3.MediaMedicamento FROM GrupoIdade3 G3 /* Idade = [32,47] */

3

In [22]:
SELECT G4.MediaMedicamento FROM GrupoIdade4 G4 /* Idade = [48,63] */

4

In [23]:
SELECT G5.MediaMedicamento FROM GrupoIdade5 G5 /* Idade = [64,80] */

5

| Grupos de Idade | Média de Medicamentos Usados |
|:---------------:|------------------------------|
|      [0,15]     |               2              |
|     [16,31]     |               2              |
|     [32,47]     |               3              |
|     [48,63]     |               4              |
|     [64,80]     |               5              |

# Navegando por Hierarquias

A seguir exercícios envolvendo navegação por hierarquias.

# Marcadores e Taxonomia

Considere o modelo para `Marcadores` e `Taxonomia`:

![UML](../images/marcadores-taxonomia-uml.png)

![Relacional](../images/marcadores-taxonomia-er.png)

Considere as sentenças SQL a seguir para montar as tabelas de `Marcadores` e `Taxonomia`:

In [24]:
DROP TABLE IF EXISTS Taxonomia;
DROP TABLE IF EXISTS Marcadores;

CREATE TABLE Taxonomia (
  Categoria VARCHAR(50) NOT NULL,
  Superior  VARCHAR(50),
  PRIMARY KEY(Categoria),
  FOREIGN KEY(Superior)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

CREATE TABLE Marcadores (
  Titulo VARCHAR(50) NOT NULL,
  Endereco VARCHAR(80) NOT NULL,
  Acessos INTEGER,
  Categoria VARCHAR(50) NOT NULL,
  PRIMARY KEY(Titulo),
  FOREIGN KEY(Categoria)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

As tabelas são preenchidas com as seguintes tuplas:

In [25]:
INSERT INTO Taxonomia VALUES ('Geral', NULL);
INSERT INTO Taxonomia VALUES ('Serviços', 'Geral');
INSERT INTO Taxonomia VALUES ('Acadêmico', 'Geral');
INSERT INTO Taxonomia VALUES ('Relacionamento', 'Geral');
INSERT INTO Taxonomia VALUES ('Busca', 'Serviços');
INSERT INTO Taxonomia VALUES ('Portal', 'Serviços');
INSERT INTO Taxonomia VALUES ('Mail', 'Serviços');
INSERT INTO Taxonomia VALUES ('Vendas', 'Serviços');
INSERT INTO Taxonomia VALUES ('Universidade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('CG', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Sociedade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Rede Social', 'Relacionamento');
INSERT INTO Taxonomia VALUES ('Marketplace', 'Vendas');
INSERT INTO Taxonomia VALUES ('Loja', 'Vendas');
INSERT INTO Taxonomia VALUES ('Notícias', 'Portal');
INSERT INTO Taxonomia VALUES ('Agregador', 'Portal');

INSERT INTO Marcadores VALUES ('Terra', 'http://www.terra.com.br', 295, 'Portal');
INSERT INTO Marcadores VALUES ('POVRay', 'http://www.povray.org', 2, 'CG');
INSERT INTO Marcadores VALUES ('SBC', 'http://www.sbc.org.br', 26, 'Sociedade');
INSERT INTO Marcadores VALUES ('Correios', 'http://www.correios.com.br', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('GMail', 'http://www.gmail.com', 296, 'Mail');
INSERT INTO Marcadores VALUES ('Google', 'http://www.google.com', 1590, 'Busca');
INSERT INTO Marcadores VALUES ('Yahoo', 'http://www.yahoo.com', 134, 'Serviços');
INSERT INTO Marcadores VALUES ('Orkut', 'http://www.orkut.com', 45, 'Rede Social');
INSERT INTO Marcadores VALUES ('iBahia', 'http://www.ibahia.com', 3, 'Portal');
INSERT INTO Marcadores VALUES ('Submarino', 'http://www.submarino.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Amazon', 'https://www.amazon.com.br', 410, 'Marketplace');
INSERT INTO Marcadores VALUES ('Americanas', 'https://www.americanas.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Mercado Livre', 'https://www.mercadolivre.com.br', 500, 'Marketplace');
INSERT INTO Marcadores VALUES ('G1', 'https://g1.globo.com', 1200, 'Notícias');
INSERT INTO Marcadores VALUES ('Folha', 'https://www.folha.uol.com.br', 850, 'Notícias');
INSERT INTO Marcadores VALUES ('Google News', 'https://news.google.com', 900, 'Agregador');
INSERT INTO Marcadores VALUES ('Flipboard', 'https://flipboard.com', 950, 'Agregador');

Conteúdo da tabela `Taxonomia`:

In [26]:
SELECT * FROM Taxonomia;

Conteúdo da tabela `Marcadores`:

In [27]:
SELECT * FROM Marcadores;

## Tarefa 1

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, sem considerar as categorias subordinadas.

In [28]:
SELECT * FROM Marcadores M
WHERE M.Categoria='Serviços'

## Tarefa 2

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, considerando as categorias subordinadas.

In [29]:
CREATE VIEW CategoriaServicos AS
SELECT DISTINCT T1.Categoria FROM Taxonomia T1, (SELECT T2.Categoria FROM Taxonomia T2 WHERE T2.Superior='Serviços') Sub
WHERE T1.Categoria='Serviços' OR T1.Categoria IN (Sub.Categoria)
                              OR T1.Superior  IN (Sub.Categoria)

In [30]:
SELECT M.Titulo,M.Endereco, M.Acessos, M.Categoria FROM Marcadores M, CategoriaServicos CS
WHERE M.Categoria=CS.Categoria